# Collaborative filtering recommender system

## Dataset description

In this assignment you will use Apache Spark to build simple movie recommendation system on big MovieLens 20M dataset (https://grouplens.org/datasets/movielens/). The dataset contains 20 million ratings for 27000 movies given by 138000 users. 

Dataset format: comma-separated values (https://en.wikipedia.org/wiki/CSV)

The first line is a header:
userId,movieId,rating,timestamp

_userId_, _movieId_ are integers representing user and movies identifiers correspondingly

_rating_ is a floating point number from 1.0 to 5.0

_timestamp_ is an integer but it won’t be used in the assignment :)

## Reading dataset

Init spark session

In [1]:
from __future__ import division, print_function, unicode_literals # For the compatibility with Python 2

In [2]:
import os

PY2 = True
python = '/usr/bin/python2' if PY2 else '/opt/conda/bin/python3.6'

os.environ['PYSPARK_PYTHON'] = os.environ['PYSPARK_DRIVER_PYTHON'] = python

In [3]:
from pyspark.sql import SparkSession

spark_session = (
    SparkSession
    .builder
    .enableHiveSupport()
    .appName("recSys")
    .master("local[8]")
    .getOrCreate()
)
# sc = spark_session.sparkContext

In [4]:
data = spark_session.read.load(
    "/data/movielens/ratings_100k.csv",
    format='com.databricks.spark.csv', 
    header=True, 
    inferSchema=True
).cache()

In [5]:
# data.show()

In this assignment you will use collaborative filtering approach for making predictions. To find latent vector representation for users and items we suggest you to use explicit ALS method. Refer to the Spark MLLib documentation for Python API details: https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html

Split dataset into three folds: on each iteration one fold will be used for testing and the other two folds will be used for training. Wrap necessary values with _Rating_ class.

In [6]:
from pyspark.mllib.recommendation import Rating

fold_count = 3

In [7]:
data.rdd.map(lambda row: Rating(row[0], row[1], row[2]))

PythonRDD[13] at RDD at PythonRDD.scala:48

In [8]:
split_df = data.randomSplit([1.] * fold_count)

Recommendations: to make your solution more efficient prepare RDDs for all the folds beforehand running the training process.

## Training ALS model

Now it is time to choose training parameters. It is recommended to set the rank equal to 20,
regularization term lambda equal to 0.01 and the number of iterations equal to 5.
Feel free to experiment and choose your own parameters to see how they influence the final score.

In [15]:
rank = 10
iterations = 15
lambda_ = 0.05

Train the explicit ALS model on two of three folds and evaluate its performance on remaining test fold. Performance evaluation should be done using the classic RMSE metric.

In [10]:
from pyspark.mllib.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

from math import sqrt

In [16]:
def rating_to_tuple(row):
    return (row.user, row.product), row.rating

def rate_als(train, test):
    train_data = train.rdd.map(lambda row: Rating(*row[:3]))
    test_data = test.rdd.map(lambda row: Rating(*row[:3]))
    # train model
    model = ALS.train(train_data, rank, iterations, lambda_)
    # make predictions
    predictions = (
        model.predictAll(
            test_data.map(lambda row: (row.user, row.product))
        ).toDF(['user', 'product', 'prediction'])
        .join(test, ((col('userId') == col('user')) & (col('movieId') == col('product'))))
        .select(
            col('user'), col('product'), col('prediction'), col('rating').alias('target'))
    )
    evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")
    return evaluator.evaluate(predictions)

In [17]:
scores = []

for i in range(fold_count):
    scores.append(rate_als(split_df[i], split_df[(i + 1) % fold_count].union(split_df[(i + 2) % fold_count])))

print (scores)

[1.1619046397350516, 1.154230985007914, 1.157638818325515]


Average RMSE scores on all the three folds and output the result to stdout.
You could refer to publicly available benchmarks (e.g. http://mymedialite.net/examples/datasets.html)
to find out what score to expect.

In [18]:
score = sum(scores) / len(scores)

In [19]:
score

1.1579248143561605